In [1]:
#Ignore 'Future Warnings'
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import os
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sklearn
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [3]:
#Set Psudeorandom Seed
seed = 42

In [4]:
model = SVC(random_state = seed, probability=True)

In [5]:
dum = pd.read_excel ('1y_offset.xlsx',index_col=0,na_values=['..'])

year_dummies = pd.get_dummies(dum.year, prefix='year').iloc[:,1:]
region_dummies = pd.get_dummies(dum.region, prefix='region').iloc[:,1:]
country_dummies = pd.get_dummies(dum.ccode, prefix='country: ').iloc[:,1:]

In [6]:
files = []

files.append(('con_1.xlsx','con_1'))
files.append(('con_2.xlsx','con_2'))
files.append(('con_3.xlsx','con_2'))

In [7]:
file2 = 'pred_2018.xlsx'
df_pred = pd.read_excel (file2,index_col=0,na_values=['..'])
#df_pred = pd.concat([df_pred, year_dummies, region_dummies, country_dummies], axis = 1)

array2 = df_pred.values
X_pred = array2[:,4:]
X_pred = StandardScaler().fit_transform(X_pred)

In [8]:
results = df_pred[['ccode']].copy()
index = results.index

In [9]:
for file, name in files:
    
    df = pd.read_excel (file,index_col=0,na_values=['..'])
    #df = pd.concat([df, year_dummies, region_dummies, country_dummies])

    #Create testing and training sets
    array = df.values

    #Create X array
    X= array[:,4:]

    # Standardizing the features
    X = StandardScaler().fit_transform(X)

    #Create Y array
    Y= array[:,3]
    Y=Y.astype('int')

    #Training/Test Split
    X_train, X_validation, Y_train, Y_validation = train_test_split(X,Y,test_size=0.2,random_state=seed)
    
    #Impliment SMOTE
    sm = SMOTE(random_state = seed) 
    X_train_smote, Y_train_smote = sm.fit_sample(X_train, Y_train)

    # Make predictions on validation dataset
    name_model = model
    name_model.fit(X_train, Y_train)
    predictions = name_model.predict(X_validation)
    print(confusion_matrix(Y_validation, predictions))
    print('')
    print('')
    pred = name_model.predict_proba(X_pred)

    df_pred = pd.DataFrame({'Con_0':pred[:,0],'Con_1': pred[:,1]}, index=index)
    results = pd.concat([results, df_pred], axis=1)

results.columns = ['Country', 'No State Based', 'State Based', 'No Nonstate', 'Nonstate', 'No One-Sided', 'One-Sided']

target = 'results.xlsx'
results.to_excel(target)
print('saved to '+target )

[[971  32]
 [ 57 184]]


[[1101   17]
 [  48   78]]


[[963  48]
 [ 73 160]]


saved to results.xlsx
